In [ ]:
import time
import datetime
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from collections import defaultdict
from nltk.corpus import stopwords

In [ ]:
# modify file name
trained_file_name = '../server/data/glove.6B.300d.txt' #'/home/jurim0301/text_purify/w2v_300.txt' 

# text dataset for word2vec training
text_file = 'pre_text' #'/home/jurim0301/text_purify/pre_text'

# modify dimensions
dims = 300

# clustering file name
file_start = 'kmeans_cluster_'

In [ ]:
# this is for pre-processing
# currently not utilized in this ipynb
def remove_stword(words, embs):
    st = stopwords.words()
    new_idx = [idx for idx in xrange(len(words)) if (words[idx] not in st)]
    print len(words), len(new_idx)
    print len(words)-len(new_idx), 'stopwords are found'
    
    return words[new_idx], embs[new_idx]

# Word2vec training

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# corpus : text corpus (list of list of words)
# dimensions : dimensions of each word representation

def word2vec_training(corpus, dimensions):
    # one sentence = one wikipedia
    start = datetime.datetime.now()
    model = Word2Vec(corpus, size=dimensions, min_count=10, window=10, workers=6)
    print str(dimensions),'training complete', datetime.datetime.now()-start

    model.save(('w2v_'+str(dimensions)+'.model'))
    model.save_word2vec_format(('w2v_'+str(dimensions)+'.txt'),fvocab='w2v_vocab.txt')


In [ ]:
# read text dataset
corpus = [line.split() for line in open(text_file,'r').read().split('\n') if len(line.split())!=0]
print len(corpus), 'documents in the text corpus'

In [ ]:
word2vec_training(corpus, dims)

# Dimension sorting

In [ ]:
import csv
df = pd.read_csv(trained_file_name, encoding='utf8', header=None, skiprows=0, quoting=3, delim_whitespace=True, index_col=[0])

In [ ]:
df.head()

In [ ]:
# remove stopwords
st = stopwords.words()
df = df[df.index.isin(st)==False]

In [ ]:
import codecs
words = df.iloc[:,0]
with codecs.open('300_concepts_no_st.txt','w') as fw:
    for i in xrange(300):
        df.sort_values(by=(i+1), ascending=False, inplace=True)
        fw.write('========================='+str(i)+'=========================\n')
        fw.write(u' '.join(df.iloc[:100,0]))
        fw.write('\n')

In [ ]:
df.sort_values(by=1, ascending=True, inplace=True)
u' '.join(df.iloc[:50, 0])

In [ ]:
df.head()
df.set_index(0)
df.head()
# df.set_index?

# K-Means clustering

In [ ]:
# input : emb (embeddings), nums (number of clusters)
# output : clustered 
def kmeans_clustering(words, embs, nums):
    
    start = datetime.datetime.now()
    kmeans = KMeans(n_clusters=nums, n_jobs=6).fit(embs)
    print 'elapsed time :', datetime.datetime.now() - start
    
    # calculate distance to the assigned cluster centroid
    cls_idx = kmeans.labels_
    centers = kmeans.cluster_centers_
    distance = np.array([np.linalg.norm((centers[cls_idx[i]]-emb)) for i,emb in enumerate(embs)])
    
    with open(file_start+str(nums)+'.txt','w') as fw:
        for i in xrange(nums):
            fw.write('========================='+str(i)+'=========================\n')
            temp = sorted([(list(words)[idx], distance[idx]) for idx in np.where(cls_idx==i)[0]], key=lambda x: x[1])[:100]
            fw.write(u' '.join([str(t[0]) for t in temp]))
            fw.write('\n')


In [ ]:
words = df.index
embs = df.iloc[:,1:].as_matrix()
nums = dims

In [ ]:

start = datetime.datetime.now()
kmeans = KMeans(n_clusters=nums ).fit(embs)
print 'elapsed time :', datetime.datetime.now() - start

# calculate distance to the assigned cluster centroid
cls_idx = kmeans.labels_
centers = kmeans.cluster_centers_
distance = np.array([np.linalg.norm((centers[cls_idx[i]]-emb)) for i,emb in enumerate(embs)])

with open(file_start+str(nums)+'.txt','w') as fw:
    for i in xrange(nums):
        fw.write('========================='+str(i)+'=========================\n')
        temp = sorted([(list(words)[idx], distance[idx]) for idx in np.where(cls_idx==i)[0]], key=lambda x: x[1])[:100]
        fw.write(u' '.join([str(t[0]) for t in temp]))
        fw.write('\n')

In [ ]:
cls_idx

In [ ]:
len(cls_idx)

In [ ]:
distance

In [ ]:
df2 = pd.DataFrame({'word': df[0],
                   'cluster': cls_idx,
                   'distance': distance })

In [ ]:
df2.head()

In [ ]:
df2.sort_values(['cluster','distance'], inplace=True)

In [ ]:
df2.to_csv('clusterDistance.csv')

In [ ]:
df2

In [ ]:
df3 = df2.set_index?

In [ ]:
df3 = df2.set_index('word')

In [ ]:
df3

In [ ]:
df3.to_csv("clusterDistance2.csv")